In [52]:
1. Install Dependencies and Setup

# This is a command in Python that uses pip, a package manager for Python, to install the TensorFlow library.

!pip install tensorflow 

#The code imports the TensorFlow library, which is a tool for creating and training machine learning models.

import tensorflow as tf

#This code imports the os module in Python, which provides a way for Python programs to interact with the operating system. 

import os

#This code uses the TensorFlow function list_physical_devices() to obtain a list of available GPUs on the machine.
#The list can be used to configure TensorFlow to utilize the GPUs for faster computation in deep learning models.

gpus = tf.config.experimental.list_physical_devices('GPU')

#This code block checks if GPUs are available and sets dynamic memory allocation for each GPU using the TensorFlow function set_memory_growth().
#This code optimizes TensorFlow to use available GPUs and improve the performance of deep learning models.

if gpus:
    try:
        # Set memory growth for each GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        # Verify memory growth is set
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

#This code obtains a list of available GPUs using TensorFlow's list_physical_devices() function 
#and configures TensorFlow to use them for faster deep learning model computation.

tf.config.list_physical_devices('GPU')

2. Remove Dodgy Images 

#This code imports the cv2 module, which allows developers to use OpenCV's image and video processing functions in Python programs.

import cv2

#The code imports the imghdr module in Python, which can be used to check the type of image file based on its content.
#It provides functions to determine the image file type or test if a file is an image file.

import imghdr

#This code creates a variable named data_dir that stores the path to a directory containing data used by a Python program.
#It makes the code more flexible and maintainable by storing the path in a variable.

data_dir = 'data'

#This code creates a list of common image file extensions used to filter files in a directory or perform other image processing tasks.

image_exts = ['jpeg', 'jpg', 'bmp', 'png']

#This code filters image files in a directory and its subdirectories.
#It reads images with OpenCV and uses imghdr to determine their types. Invalid files are removed with os.remove().

for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

3.Load Data

#This code imports the pyplot module from the matplotlib package and renames it to plt. pyplot provides functions for creating and customizing plots and renaming.

from matplotlib import pyplot as plt

#Code imports numpy package. Numpy is used for working with numerical data arrays and provides many math functions.

import numpy as np

#This code uses image_dataset_from_directory() from tf.keras.utils to load and preprocess image data from a directory for use in machine learning models.
#It automatically splits the data into training and validation sets and returns a tf.data.Dataset object containing batches of images and labels.

data = tf.keras.utils.image_dataset_from_directory('data')

#This code creates a numpy iterator over a tf.data.Dataset object, which enables the user to iterate over the dataset and convert it to numpy format. 
#This can be useful for operations like visualization or data manipulation, but may not be suitable for large datasets.

data_iterator = data.as_numpy_iterator()

##This code uses next() to get the next batch of data and labels from a tf.data.Dataset object via a numpy iterator created using the as_numpy_iterator() method.

batch = data_iterator.next()

#This code prepares the image data for deep learning by creating a visualization of the data and normalizing the pixel values.

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

4. Scale Data

#This code applies a lambda function to the data using the map() method of the tf.data.Dataset object.
#The lambda function normalizes the pixel values of the images by dividing them by 255, which scales the pixel values to the range [0,1].

data = data.map(lambda x,y: (x/255, y))

#This code returns the first batch of data and labels from the tf.data.Dataset object as numpy arrays using the as_numpy_iterator() method and next() function.

data.as_numpy_iterator().next()

5. Split Data

#This code defines the sizes of the training, validation, and test sets as a percentage of the total size of the data.
#The training set is 70% of the data, the validation set is 20%, and the test set is 10%.

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

#train_size is a variable that represents the size of the training dataset, which is calculated by taking 70% of the total length of the original dataset.

train_size

#The code splits a dataset into three subsets: training, validation, and testing, based on the sizes specified. 
#The training set is taken from the beginning of the dataset, followed by the validation set, and finally the test set. 

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

6. Build Deep Learning Model

#The 'train' dataset contains 70% of the preprocessed data and is intended for training a machine learning model.
#The code splits the data into three subsets, allowing the model to be trained on one subset while being evaluated on the others.

train

#Sequential is a class in tensorflow.keras.models for building a linear stack of layers in a deep learning model.

from tensorflow.keras.models import Sequential

#Conv2D, MaxPooling2D, Dense, Flatten, and Dropout are classes in tensorflow.keras.layers used for creating convolutional, max pooling, \
#fully connected, flattening, and dropout layers, respectively. 


from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

#The code initializes an instance of the Sequential class in Keras and assigns it to the variable "model".

model = Sequential()

#The code creates a CNN for image classification using a popular architecture.
#It has three Convolutional layers with max pooling, a Flatten layer, two fully connected layers, and an output layer. ReLU and sigmoid activation functions are used.

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))


#This code compiles the Keras model by specifying the optimizer, loss function, and evaluation metrics to be used during training. 
#The optimizer used is 'adam', which is a popular optimization algorithm.
#The loss function used is Sparse Categorical Cross-entropy, which is often used for multi-class classification tasks. 

from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])


#model.summary() provides a summary of the layers in a Keras model, including the layer type, output shape, and number of parameters.

model.summary()

7. Train

#The code trains a Keras machine learning model with training and validation data using the fit() method, and includes a callback to TensorBoard for monitoring training progress. 
#The output is a History object containing information about the training process.

logdir='logs'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

8. Plot Performance

#The code plots the training and validation loss of a machine learning model over epochs.

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

#The code plots the training and validation accuracy over epochs.

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

9. Evaluate

#Precision, Recall, and BinaryAccuracy are evaluation metrics for binary classification models that measure the model's ability to avoid false positives, detect positive cases, and accuracy rate

from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

#Precision, Recall, and BinaryAccuracy are metrics used to evaluate classification models. They are imported from the tensorflow.keras.metrics library

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

#This code calculates the test accuracy of a trained model using the CategoricalAccuracy metric. 
#It loops through the test dataset batches and updates the metric object with the ground truth and predicted labels.

from tensorflow.keras.metrics import CategoricalAccuracy

# create metric objects
categorical_accuracy = CategoricalAccuracy()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    categorical_accuracy.update_state(y, yhat)

print(f'Test Categorical Accuracy: {categorical_accuracy.result().numpy()}')


#This will print the precision, recall and binary accuracy metrics of the trained model on the test dataset.

print(pre.result(), re.result(), acc.result())

10. Test

import cv2
import numpy as np
import matplotlib.pyplot as plt

#This code reads an image file and displays it using matplotlib.
#If the image is read successfully, it is converted from BGR to RGB using OpenCV's cv2.cvtColor() function, and then displayed using plt.imshow() and plt.show().
#If the image cannot be read, the code prints a message indicating that it could not read the image.

img = cv2.imread('6.jpg')
if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = np.asarray(img)
    plt.imshow(img_array)
    plt.show()
else:
    print('Could not read image')

#The code resizes an input image to a new size of (256, 256) using the tf.image.resize() function, then displays the resulting image using plt.imshow().

resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

#This code uses a pre-trained model to predict the class of an input image by taking the argmax of the predicted class probabilities and retrieving the class name from a list of class names.

class_names = ["ABBOTTS BABBLER", "ABYSSINIAN GROUND HORNBILL", "BALD EAGLE"]
yhat = model.predict(np.expand_dims(resize / 255, 0))

# get predicted class index
class_idx = tf.argmax(yhat, axis=1)[0]

predicted_class = class_names[class_idx]
print(f"Predicted class is {predicted_class}")


